In [ ]:
import configparser

config = configparser.ConfigParser()
config.read('config.ini')

apikey = config['DEFAULT']['ApiKey']
url = config['DEFAULT']['Url']
field_selection = 'text_entry'

In [ ]:
import json
import sys

try:
    import pandas as pd
except ImportError:
    subprocess.run([sys.executable, "-m", "pip", "install", "pandas"], check=True)
finally:
    import pandas as pd

try:
    from ibm_watson import LanguageTranslatorV3
    from ibm_cloud_sdk_core.authenticators import IAMAuthenticator
except ImportError:
    subprocess.run([sys.executable, "-m", "pip", "install", "ibm-watson"], check=True)
    subprocess.run([sys.executable, "-m", "pip', 'install', 'ibm-cloud-sdk-core"], check=True)
finally:
    from ibm_watson import LanguageTranslatorV3
    from ibm_cloud_sdk_core.authenticators import IAMAuthenticator

authenticator = IAMAuthenticator(apikey)
translator = LanguageTranslatorV3(
    version='2018-05-01',
    authenticator=authenticator)

translator.set_service_url(url)

In [ ]:
models = translator.list_models().get_result()
print(json.dumps(models, indent=2))

In [ ]:
data = pd.read_csv('test.csv')
print(data)

text_entries = data[field_selection].to_list()
print(text_entries)

In [ ]:
fr_en_translation_result = translator.translate(
    text=text_entries,
    model_id='fr-CA-en').get_result()
print(json.dumps(fr_en_translation_result, indent=2))

In [ ]:
fr_en_translations = []
for line in fr_en_translation_result["translations"]:
    fr_en_translations.append(line["translation"])
print(fr_en_translations)

In [ ]:
en_fr_translation_result = translator.translate(
    text=text_entries,
    model_id='en-fr-CA').get_result()
print(json.dumps(en_fr_translation_result, indent=2))

In [ ]:
en_fr_translations = []
for line in en_fr_translation_result["translations"]:
    en_fr_translations.append(line["translation"])
print(en_fr_translations)

In [ ]:
data['fr-CA-en_result'] = fr_en_translations
data['en-fr-CA_result'] = en_fr_translations
data.to_csv('translated_result.csv', index=False)
print(data)